In [45]:
# import the necessary packages
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.ops as ops
from net import Net
import cv2
import random
import time

import warnings
#suppress warning
warnings.filterwarnings('ignore')

In [46]:
face_detection_net = Net()
face_detection_net.load_state_dict(torch.load("./saved_model.pth",map_location=torch.device('cuda')))

<All keys matched successfully>

In [47]:
image_path = "./face_detection_images/detection_test1.jpg"
image = cv2.imread(image_path)

In [48]:
# initialize OpenCV's selective search implementation and set the
# input image
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)

In [49]:
print("[INFO] using *quality* selective search")
#ss.switchToSelectiveSearchQuality()
ss.switchToSelectiveSearchQuality()

[INFO] using *quality* selective search


In [50]:
start = time.time()
rects = ss.process()
end = time.time()

print(len(rects))

31116


In [51]:
transform = transforms.Compose([ 
    transforms.ToPILImage(),
    transforms.Grayscale(),
    transforms.ToTensor(),    
    transforms.Resize((36,36))
])

In [52]:
(H, W) = image.shape[:2]

rois = []
locs = []

for (x, y, w, h) in rects:
	# if the width or height of the region is less than 10% of the
	# image width or height, ignore it (i.e., filter out small
	# objects that are likely false-positives)
	if w / float(W) < 0.05 or h / float(H) < 0.05:
		continue
	# extract the region from the input image, convert it from BGR to
	# RGB channel ordering
	roi = image[y:y + h, x:x + w]
	roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

	# further preprocess by the ROI

	roi_tensor_gray = transform(roi)
	
	# update our list of ROIs and associated coordinates
	rois.append(roi_tensor_gray)
	locs.append((x, y, x + w, y + h))

In [53]:
print(len(rois))
print(len(locs))

13490
13490


In [54]:
stacked_tensor = torch.stack(rois, dim=0)

print(stacked_tensor.size())

output = face_detection_net(stacked_tensor)

torch.Size([13490, 1, 36, 36])


In [55]:
probs = torch.nn.functional.softmax(output, dim=1)
probs_list = probs.tolist()

labels = {'valid_probs': [],
          'boxes': []}

In [56]:
for i in range(0,len(probs_list)):
    if (probs_list[i][1] >= 0.95):
        box = locs[i]

        labels['valid_probs'].append(probs_list[i][1])
        labels['boxes'].append(box)

tensor_boxes = torch.Tensor(labels['boxes'])
tensor_probs = torch.Tensor(labels['valid_probs'])

valid_box = ops.nms(tensor_boxes, tensor_probs, iou_threshold=0.1)
        

In [57]:
print(labels['boxes'])

print(valid_box)

[(1164, 423, 1242, 504), (1149, 363, 1236, 494), (352, 69, 424, 196), (1129, 209, 1201, 305), (702, 260, 776, 376), (124, 200, 198, 294), (1151, 378, 1236, 482), (699, 240, 774, 377), (120, 201, 199, 294), (411, 397, 484, 485), (1148, 361, 1244, 535), (118, 201, 198, 303), (120, 199, 198, 293), (121, 201, 198, 296), (805, 106, 889, 205), (1207, 97, 1280, 197), (1209, 98, 1287, 202), (402, 397, 485, 488), (194, 382, 276, 491), (184, 390, 267, 495), (172, 92, 245, 179), (1149, 363, 1243, 537), (125, 197, 197, 292), (1208, 95, 1286, 243), (1008, 90, 1086, 176), (1207, 97, 1287, 197), (394, 419, 471, 523), (172, 70, 245, 179), (694, 243, 772, 382), (699, 240, 772, 376), (1149, 379, 1236, 482), (337, 72, 410, 161), (1129, 205, 1201, 302), (172, 92, 245, 177), (882, 202, 967, 290), (1151, 361, 1244, 540), (1145, 374, 1241, 541), (1149, 374, 1233, 482), (1148, 361, 1242, 535), (809, 106, 881, 192), (696, 253, 778, 377), (1161, 432, 1243, 504), (698, 239, 772, 376), (1130, 201, 1204, 306), (11

In [58]:
 
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 1 px 
thickness = 1

img_color = cv2.imread(image_path)
  
# Using cv2.rectangle() method 
# Draw a rectangle with blue line borders of thickness of 1 px 
for index in valid_box:
    box = labels['boxes'][index]
    (x,y,z,t) = box
    cv2.rectangle(img_color, (x,y), (z,t), color, thickness)


cv2.imshow('image', img_color)

# add wait key. window waits until user presses a key
cv2.waitKey(0)
# and finally destroy/close all open windows
cv2.destroyAllWindows()

In [59]:
### DEBUGGING
# # loop over the region proposals in chunks (so we can better
# # visualize them)
# for i in range(0, len(rects), 100):
# 	# clone the original image so we can draw on it
# 	output = image.copy()
# 	# loop over the current subset of region proposals
# 	for (x, y, w, h) in rects[i:i + 100]:
# 		# draw the region proposal bounding box on the image
# 		color = [random.randint(0, 255) for j in range(0, 3)]
# 		cv2.rectangle(output, (x, y), (x + w, y + h), color, 2)
# 	# show the output image
# 	cv2.imshow("Output", output)
# 	key = cv2.waitKey(0) & 0xFF
# 	# if the `q` key was pressed, break from the loop
# 	if key == ord("q"):
# 		break

KeyboardInterrupt: 